### Installation

In [1]:
%%capture
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
!pip install --no-deps cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
!pip install --no-deps unsloth
!pip install wandb
pip install unsloth
pip install --upgrade --force-reinstall numpy==1.23.5 pandas

In [1]:
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
import wandb
import os
from datasets import load_dataset

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.581 GB.
0.0 GB of memory reserved.


In [3]:

max_seq_length = 2048
dtype = None 
load_in_4bit = True 

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/DeepSeek-R1-Distill-Llama-8B-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    
)

==((====))==  Unsloth 2025.2.12: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    GPU: Tesla T4. Max memory: 14.581 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 8, 
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, 
    bias = "none",    
    use_gradient_checkpointing = "unsloth", 
    random_state = 3407,
    use_rslora = False,  
    loftq_config = None, 
)

Unsloth 2025.2.12 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [5]:
# Load dataset and apply formatting function
dataset = load_dataset("nbertagnolli/counsel-chat", split="train")


Repo card metadata block was not found. Setting CardData to empty.


In [6]:
print(dataset.column_names)

['questionID', 'questionTitle', 'questionText', 'questionLink', 'topic', 'therapistInfo', 'therapistURL', 'answerText', 'upvotes', 'views']


In [7]:
# Define the prompt template
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

# Get EOS token
EOS_TOKEN = tokenizer.eos_token  # Ensure the tokenizer is initialized

# Define the function to process your dataset
def formatting_prompts_func(examples):
    instruction = "You are an AI CBT therapist. Respond appropriately in the following conversation:"
    inputs = examples["questionText"]  # Using questionText as Input
    outputs = examples["answerText"]   # Using answerText as Response

    texts = [
        alpaca_prompt.format(instruction, inp, f"<think>Analyzing the patient's thoughts...</think>\n{out}") + EOS_TOKEN
        for inp, out in zip(inputs, outputs)
    ]
    
    return {"text": texts}

dataset = dataset.map(formatting_prompts_func, batched=True)

# Display the first formatted sample
print(dataset[0])

Map:   0%|          | 0/2775 [00:00<?, ? examples/s]

{'questionID': 0, 'questionTitle': 'Do I have too many issues for counseling?', 'questionText': 'I have so many issues to address. I have a history of sexual abuse, I’m a breast cancer survivor and I am a lifetime insomniac.    I have a long history of depression and I’m beginning to have anxiety. I have low self esteem but I’ve been happily married for almost 35 years.\n   I’ve never had counseling about any of this. Do I have too many issues to address in counseling?', 'questionLink': 'https://counselchat.com/questions/do-i-have-too-many-issues-for-counseling', 'topic': 'depression', 'therapistInfo': 'Jennifer MolinariHypnotherapist & Licensed Counselor', 'therapistURL': 'https://counselchat.com/therapists/jennifer-molinari', 'answerText': 'It is very common for\xa0people to have multiple issues that they want to (and need to) address in counseling.\xa0 I have had clients ask that same question and through more exploration, there is often an underlying fear that they\xa0 "can\'t be h

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [8]:


trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, 
    args = TrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 4,
        warmup_steps = 1,
        max_steps = 50,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Applying chat template to train dataset (num_proc=2):   0%|          | 0/2775 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/2775 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/2775 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [9]:
trainer_stats = trainer.train()

# Save the fine-tuned model
model.save_pretrained_gguf("few shot learning - deepmental", tokenizer, quantization_method="q8_0")

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 2,775 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 4
\        /    Total batch size = 16 | Total steps = 50
 "-____-"     Number of trainable parameters = 20,971,520
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ibrahimshaban1994 (ibrahimshaban1994-university-of-london) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss
1,3.306700
2,3.452100
3,3.214600
4,3.038900
5,2.685700
6,2.607800
7,2.556100
8,2.451100
9,2.280000
10,2.143100


Attempted to log scalar metric loss:
3.3067
Attempted to log scalar metric grad_norm:
1.3190186023712158
Attempted to log scalar metric learning_rate:
0.0002
Attempted to log scalar metric epoch:
0.005763688760806916
Attempted to log scalar metric loss:
3.4521
Attempted to log scalar metric grad_norm:
1.3312839269638062
Attempted to log scalar metric learning_rate:
0.0001959183673469388
Attempted to log scalar metric epoch:
0.011527377521613832
Attempted to log scalar metric loss:
3.2146
Attempted to log scalar metric grad_norm:
1.578161358833313
Attempted to log scalar metric learning_rate:
0.00019183673469387756
Attempted to log scalar metric epoch:
0.01729106628242075
Attempted to log scalar metric loss:
3.0389
Attempted to log scalar metric grad_norm:
1.8909801244735718
Attempted to log scalar metric learning_rate:
0.00018775510204081634
Attempted to log scalar metric epoch:
0.023054755043227664
Attempted to log scalar metric loss:
2.6857
Attempted to log scalar metric grad_norm:
1

 50%|█████     | 16/32 [00:01<00:01, 15.83it/s]
We will save to Disk and not RAM now.
100%|██████████| 32/32 [00:46<00:00,  1.44s/it]

RuntimeError: Unsloth: The file 'llama.cpp/llama-quantize' or 'llama.cpp/quantize' does not exist.
But we expect this file to exist! Maybe the llama.cpp developers changed the name?